In [ ]:
planet1 = 'earth'
planet2 = 'venus'

planet1, planet2 = planet2, planet1

In [ ]:
print(planet1, planet2)

In [ ]:
#Scraping

import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Prepare CSV file
# csvFile = open("tripadvisor_reviews.csv", "w", newline='', encoding="utf-8")
# csvWriter = csv.writer(csvFile)
# csvWriter.writerow(['Date', 'Title', 'Review'])

#url = "https://www.tripadvisor.com/Hotel_Review-g304026-d312097-Reviews-Eko_Hotels_Suites-Lagos_Lagos_State.html"
url = "https://www.tripadvisor.com/Hotel_Review-g304026-d312097-Reviews-or400-Eko_Hotels_Suites-Lagos_Lagos_State.html#REVIEWS"
driver = webdriver.Chrome()

driver.get(url)

# Initialize the current_url variable
#current_url = driver.current_url

count = 1
rev = 1
# Loop to scrape reviews from multiple pages
while True:
    count += 1
    page_source = driver.page_source
    #print(current_url)
    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Extract review elements
    review_elements = soup.find_all('span', class_='QewHA')

    # Extract and write the reviews to the CSV
    for review_element in review_elements:
        review_text = review_element.text
        print(f"{rev}" + "  " + review_text + "\n")
        rev +=1
        # Write to CSV
        # csvWriter.writerow(['Date', 'Title', review_text])

    # Check if there is a "Next" button for pagination
    next_button = driver.find_elements(By.CLASS_NAME, 'ui_button.nav.next.primary ')

    if count > 3:
        break

    # If there's a "Next" button, click it to go to the next page
    if next_button:
        is_disabled = "disabled" in next_button[0].get_attribute("class")
        if is_disabled:
            break

        next_button[0].click()
        # Wait for the next page to load by checking the URL change
        WebDriverWait(driver, 20).until(EC.url_changes(current_url))
        time.sleep(10)
        #current_url = driver.current_url  # Update the current URL
        #driver.get(current_url)
    else:
        break

# csvFile.close()
driver.quit()



In [4]:
# import csv

from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
driver = webdriver.Edge()

url = "https://www.tripadvisor.com/Hotel_Review-g304026-d312097-Reviews-Eko_Hotels_Suites-Lagos_Lagos_State.html"
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
                     
# Prepare CSV file
# csvFile = open("html1.csv", "w", newline='', encoding="utf-8")
# csvWriter = csv.writer(csvFile)
# csvWriter.writerow([html])
# csvFile = open("soup1.csv", "w", newline='', encoding="utf-8")
# csvWriter = csv.writer(csvFile)
# csvWriter.writerow([soup])
# print("Done")


# Extract the information
review_div = soup.find('div', class_='YibKl MC R2 Gi z Z BB pBbQr')
if review_div:
    # Extract the review title
    review_title = review_div.find('div', class_='KgQgP MC _S b S6 H5 _a')
    if review_title:
        title = review_title.text.strip()
        print(f"Review Title: {title}")

    # Extract the review text
    review_text_div = review_div.find('div', class_='QewHA H4 _a')
    if review_text_div:
        review_text = review_text_div.select_one(':first-child').text.strip()
        print(f"Review Text: {review_text}")

    # Extract the stay date
    stay_date_span = review_div.find('span', class_='usajM')
    if stay_date_span:
        stay_date = stay_date_span.next_sibling.strip()
        print(f"Stay Date: {stay_date}")

    # Extract the trip type
    trip_type_span = review_div.find('span', class_='trip_type_label')
    if trip_type_span:
        trip_type = trip_type_span.next_sibling.strip()
        print(f"Trip Type: {trip_type}")

    # Extract the room tips
    room_tip_div = review_div.find('div', class_='tkWaG b')
    if room_tip_div:
        room_tip = room_tip_div.next_sibling.strip()
        print(f"Room Tips: {room_tip}")

    # Extract contributions
    author_div = review_div.find('div', class_='MziKN')
    if author_div:
        author_info = author_div.text.strip()
        print(f"Author Info: {author_info}")

    # Extract helpful votes
    author_div = review_div.find('div', class_='MziKN')
    if author_div:
        author_info = author_div.text.strip()
        print(f"Author Info: {author_info}")

    # Extract helpful votes, contributions, and review author
    author_div = review_div.find('div', class_='MziKN')
    if author_div:
        author_info = author_div.text.strip()
        print(f"Author Info: {author_info}")

    # Extract the review rating
    rating_div = review_div.find('div', class_='Hlmiy F1')
    if rating_div:
        rating_span = rating_div.find('span', class_='ui_bubble_rating')
        if rating_span:
            rating_class = rating_span['class'][1].replace('bubble_', '') / 10
            print(f"Rating Class: {rating_class}")




driver.quit()


# csvFile.close()


Review Title: AC
Stay Date: 
Trip Type: Traveled solo
Rating Class: bubble_50


In [3]:

import csv
import requests

from bs4 import BeautifulSoup

url = "https://www.tripadvisor.com/Hotel_Review-g304026-d312097-Reviews-Eko_Hotels_Suites-Lagos_Lagos_State.html"
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
                     
# Prepare CSV file
csvFile = open("html2.csv", "w", newline='', encoding="utf-8")
csvWriter = csv.writer(csvFile)
csvWriter.writerow([html])
csvFile = open("soup2.csv", "w", newline='', encoding="utf-8")
csvWriter = csv.writer(csvFile)
csvWriter.writerow([soup])
print("Done")


#driver.quit()


# Extract and print the reviews
# for review_element in review_elements:
#     review_text = review_element.text
#     print(review_text + "\n")

csvFile.close()


